In [16]:
#テキストの属性をもとに段落を判別しつつテキストデータを取得
import fitz  # PyMuPDF
import re

# テキストクリーニング関数
def clean_text_by_attributes(page, header_y=None, footer_y=None, left_margin_x=None, right_margin_x=None):
    text = ""
    blocks = page.get_text("dict")["blocks"]
    previous_font = None
    previous_size = None
    previous_color = None
    previous_bbox = None
    previous_text_type = None

    def get_threshold_by_font_size(font_size):
        # フォントサイズによって閾値を動的に変更
        return font_size * 2.2

    for block in blocks:
        if "lines" not in block:
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                font = span["font"]
                size = span["size"]
                color = span["color"]
                bbox = span["bbox"]  # [x0, y0, x1, y1]
                text_type = span.get("text_type", "text")  # デフォルトで"text"
                content = span["text"]

                # テキスト位置によるフィルタリング
                if bbox[3] is not None and  bbox[3] < header_y:  # ヘッダ・フッタの除外
                    #ヘッダ領域：bbox[3]: 下端のY座標 (y1) < header_y
                    continue
                if bbox[1] is not None and bbox[1] > footer_y:  # ヘッダ・フッタの除外
                    #フッタ領域：bbox[1]: 上端のY座標 (y0) > footer_y
                    continue
                if bbox[2] is not None and bbox[2] < left_margin_x:  # 左右マージンの除外
                    #レフトマージン領域：bbox[2]: 右端のX座標 (x1) < left_margin_x
                    continue
                if bbox[0] is not None and bbox[0] > right_margin_x:  # 左右マージンの除外
                    #ライトマージン領域：bbox[0]: 左端のX座標 (x0) > right_margin_x
                    continue

                # \nの直前にある半角スペースを除外
                content = re.sub(r" \n", "\n", content)
                content = re.sub(r"^ ", "", content)

                threshold = get_threshold_by_font_size(size)  # フォントサイズに基づく閾値

                # フォント、サイズ、色、位置、幅、タイプのいずれかが変化した場合は段落の区切りと見なす
                if (
                    (previous_bbox and abs(bbox[3] - previous_bbox[3]) > threshold) or  # 縦位置の大きな変化
                    previous_text_type != text_type
                ):
                    text += "\n\n"  # 新しい段落の開始
                else:
                    text += ""  # 同じ段落と見なして連結
                    #text += "■"  # 連結位置確認用

                text += content
                previous_font = font
                previous_size = size
                previous_color = color
                previous_bbox = bbox
                previous_text_type = text_type

    # 最終的な改行の調整
    text = re.sub(r"\n{3,}", "\n\n", text).strip()
    return text


# PDFファイルのパス
base_dir = "ir/2025/"
#base_dir = "ir/2024/"
pdf_dir =  "34070_旭化成"
pdf_filename = "24jp.pdf"
#pdf_dir =  '41880_三菱ケミカルグループ'
#pdf_filename = "23.pdf"

pdf_path = base_dir + pdf_dir + "/" + pdf_filename

# PDFファイルのオープン
doc = fitz.open(pdf_path)

# フィルタリング範囲の指定（適宜調整）
#旭化成
header_y = 37  # ヘッダ除外のbbox_y1閾値（ヘッダ範囲内の最大値）
footer_y = None  # フッタ除外のbbox_y0閾値（フッタ範囲内の最小値）
left_margin_x = None  # 左マージン除外のbbox_x1閾値（範囲内の最大値）
right_margin_x = None  # 右マージン除外のbbox_x0閾値（範囲内の最小値）

#三菱ケミカル
#header_y = 24  # ヘッダ除外のbbox_y1閾値（ヘッダ範囲内の最大値）
#footer_y = 652  # フッタ除外のbbox_y0閾値（フッタ範囲内の最小値）
#left_margin_x = 174  # 左マージン除外のbbox_x1閾値（範囲内の最大値）
#right_margin_x = 997  # 右マージン除外のbbox_x0閾値（範囲内の最小値）

#レゾナック
#header_y = 26  # ヘッダ除外のbbox_y1閾値（ヘッダ範囲内の最大値）
#footer_y = 651  # フッタ除外のbbox_y0閾値（フッタ範囲内の最小値）
#left_margin_x = None  # 左マージン除外のbbox_x1閾値（範囲内の最大値）
#right_margin_x = None  # 右マージン除外のbbox_x0閾値（範囲内の最小値）

# PDFからテキストを抽出し、クリーニング
cleaned_text = ""

for page_number, page in enumerate(doc, start=0):
    # ページ開始タグ
    cleaned_text += f"<page {page_number}>\n"
    # ページ内容
    cleaned_text += clean_text_by_attributes(page, header_y, footer_y, left_margin_x, right_margin_x) + "\n"
    # ページ終了タグ
    cleaned_text += f"</page {page_number}>\n\n"

# 出力ファイルパス
output_file_path = pdf_path[:len(pdf_path)-4] + ".txt"

# ファイルへの書き込み
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print(f"ファイルが保存されました: {output_file_path}")



ファイルが保存されました: ir/2025/34070_旭化成/24jp.txt
